In [6]:
# Pythonでやる必要はないがせっかくなので
# 60. KVSの構築
# PyPi: https://pypi.python.org/pypi/redis
# Redisのデータ構造: http://qiita.com/xuj/items/8de57e4bec8a36aa64a6
import redis
import json

redis_db = redis.StrictRedis(host="localhost", port=6379, db=0)
with open("./artist.json", "r") as f:
    artists = [json.loads(artist) for artist in f]

In [59]:
# かなり時間がかかるのでpipelineクラスを使うべき
with redis_db.pipeline() as pipe:
    for artist in artists:
        pipe.set(artist["name"], artist["area"] if "area" in artist else "")
    pipe.execute()

In [60]:
# 61. KVSの検索

artist_area = redis_db.get("The Silhouettes")
print(artist_area)

b'Netherlands'


In [61]:
# 62. KVS内の反復処理

with redis_db.pipeline() as pipe:
    for key in redis_db.keys("*"):
        pipe.get(key)
    artist_areas = pipe.execute()
    print(len([area for area in artist_areas if area == b"Japan"]))

21946


In [62]:
# 63. オブジェクトを値に格納したKVS
# PythonでList型のデータをjson.dumpsしてRedisに保存してもいいが、できるだけRedisの型に従う
# 今回はRedisのList型を使用する
import json

redis_db.flushall()
with redis_db.pipeline() as pipe:
    for artist in artists:
        if "tags" in artist:
            for tag in artist["tags"]:
                pipe.lpush(artist["name"], json.dumps(tag))
    pipe.execute()

In [63]:
tags = redis_db.lrange("Masaru Narita & D.K.I.", 0, -1)
for tag in tags:
    tag = json.loads(tag.decode("utf-8"))
    print("count: " + str(tag["count"]) + ", value: " + tag["value"])

count: 1, value: split me up


In [85]:
# 64. MongoDBの構築
#  公式Tutorial: https://api.mongodb.com/python/current/tutorial.html
# リモートからの接続を許可する場合は/etc/mongod.confを修正する
from pymongo import MongoClient

client = MongoClient('localhost', 27017)
# DBの初期化
client.drop_database("artists")
db = client.artists
mongo_artists = db.artists
results = mongo_artists.insert_many(artists)
print(len(results.inserted_ids))

921337


In [89]:
# indexの作成
import pymongo

result = mongo_artists.create_index([("name", pymongo.ASCENDING)])
result = mongo_artists.create_index([("aliases.name", pymongo.ASCENDING)])
result = mongo_artists.create_index([("tags.value", pymongo.ASCENDING)])
result = mongo_artists.create_index([("rating.value", pymongo.ASCENDING)])

In [90]:
# 65. MongoDBの検索
result = mongo_artists.find_one({"name": "Queen"})
print(result)

{'gender': 'Female', 'id': 701492, 'type': 'Character', 'name': 'Queen', 'area': 'Japan', 'gid': '420ca290-76c5-41af-999e-564d7c71f1a7', '_id': ObjectId('57af33d8182ade6c58db43b8'), 'sort_name': 'Queen', 'ended': True, 'tags': [{'count': 1, 'value': 'kamen rider w'}, {'count': 1, 'value': 'related-akb48'}], 'aliases': [{'name': 'Queen', 'sort_name': 'Queen'}]}


In [92]:
# 66. 検索件数の取得
results = mongo_artists.find({"area": "Japan"})
print(len([artist for artist in results]))

22821


In [93]:
# 67. 複数のドキュメントの取得
results = mongo_artists.find({"aliases.name": "Queen"})
for artist in results:
    print(artist)

{'gender': 'Female', 'id': 701492, 'type': 'Character', 'name': 'Queen', 'area': 'Japan', 'gid': '420ca290-76c5-41af-999e-564d7c71f1a7', '_id': ObjectId('57af33d8182ade6c58db43b8'), 'sort_name': 'Queen', 'ended': True, 'tags': [{'count': 1, 'value': 'kamen rider w'}, {'count': 1, 'value': 'related-akb48'}], 'aliases': [{'name': 'Queen', 'sort_name': 'Queen'}]}


In [96]:
# 68. ソート
high_rating_artists = [artist for artist in mongo_artists.find({"tags.value": "dance"}) if "rating" in artist]
sorted_artists = sorted(high_rating_artists, key=lambda x:x["rating"]["count"], reverse=True)
for artist in sorted_artists[:10]:
    print(artist["name"] + "\t" + str(artist["rating"]["count"]))

Madonna	26
Björk	23
The Prodigy	23
Rihanna	15
Britney Spears	13
Maroon 5	11
Adam Lambert	7
Fatboy Slim	7
Basement Jaxx	6
Cornershop	5


In [97]:
# 69. Webアプリケーションの作成
# アーティスト名、アーティスト別名による検索を行うWebアプリケーションを実装した
# タグによる検索、レーティングによるソートは行わない